In [83]:
import re
import time
import random
import prettytable as pt
from bs4 import BeautifulSoup

<h3>Choose world's BIGGEST shopping website - Amazon to crawl</h3>

<h4>Login Script</h4>

In [61]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from settings import LOGIN_URL, TIMEOUT, EMAIL, PASSWORD

def login_to_amazon():
    """Login to Amazon."""
    # login to Amazon

    options = Options()
    options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_2) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1309.0 Safari/537.17")
    options.add_argument("--start-maximized")  # Maximize the browser window
    options.add_argument("--disable-notifications")
    options.add_argument("--disable-popup-blocking")
    options.add_argument("--disable-infobars")
    options.add_argument("--disable-extensions")
    options.add_experimental_option("prefs", 
                                {"profile.default_content_setting_values.notifications": 2 
                                }) 

    driver = webdriver.Chrome(
    # WEB_DRIVER_LOCATION, 
    options=options)
    driver.get(LOGIN_URL)
    time.sleep(TIMEOUT)
    login = driver.find_element(By.ID, 'ap_email')
    login.send_keys(EMAIL)
    login.send_keys(Keys.ENTER)

    submit = driver.find_element(By.ID, 'ap_password')
    submit.send_keys(PASSWORD)
    submit.send_keys(Keys.ENTER)

### Extract Data

Eight key points for every Products are:
1. Book Title
2. Product URL
3. Price
4. Solder
5. Review
6. Review Number
7. Image URL
8. Crawl Time

In [62]:
def get_product_title(item):
    product_title = item.find('span', class_='a-size-medium a-color-base a-text-normal')
    if product_title:
        return product_title.text.strip()
    else:
        return "<missing product title>"

In [63]:
BASE_URL = 'https://www.amazon.com'
def get_product_url(item):
    product_url = item.find('a', class_='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
    if product_url and product_url.get('href'):
        # if the url is relative, add the base url
        if product_url.get('href').startswith('/'):
            return BASE_URL+product_url.get('href').strip()
        return product_url.get('href').strip()
    else:
        return "<missing product url>"

In [64]:
def get_product_price(item):
    product_price = item.find('span', class_='a-offscreen')
    if product_price:
        return product_price.text.strip()
    else:
        return "<missing product price>"

In [65]:
def get_product_solder(item):
    product_solder = item.find('a', class_='a-size-base a-link-normal s-underline-text s-underline-link-text s-link-style')
    if product_solder:
        return product_solder.text.strip()
    else:
        return "<missing product solder>"

In [66]:
def get_product_review(item):
    product_review = item.find('span', class_='a-icon-alt')
    if product_review:
        return product_review.text.strip()
    else:
        return "<missing product review>"

In [67]:
def get_product_review_number(item):
    product_review_number = item.find('span', class_='a-size-base s-underline-text')
    if product_review_number:
        return product_review_number.text.strip()
    else:
        return "<missing product review number>"

In [68]:
BASE_URL = 'https://www.amazon.com'
def get_product_image_url(item):
    product_image_url = item.find('img', class_='s-image')
    if product_image_url and product_image_url.get('src'):
        if product_image_url.get('src').startswith('/'):
            return BASE_URL+product_image_url.get('src').strip()
        return product_image_url.get('src').strip()
    else:
        return "<missing product image url>"

<h4>extract information from product item</h4>

In [69]:
def product_extractor(item):
    crawl_time = time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time()))
    product_title = get_product_title(item)
    product_url = get_product_url(item)
    product_price = get_product_price(item)
    product_solder = get_product_solder(item)
    product_review = get_product_review(item)
    product_review_number = get_product_review_number(item)
    product_image_url = get_product_image_url(item)

    return (product_title, product_url, product_price, product_solder, product_review, product_review_number, product_image_url, crawl_time)       

<h4>Get next page of amazon products</h4>

In [70]:
# get next page url
def get_next_page_url(soup):
    BASE_URL = 'https://www.amazon.com'
    # find the element that contains the next page url
    urlListElement = soup.find('a', {'class': 's-pagination-item s-pagination-next s-pagination-button s-pagination-separator'})
    if urlListElement and urlListElement.get('href'):
        # if the url is relative, add the base url
        if urlListElement.get('href').startswith('/'):
            next_url = BASE_URL+urlListElement.get('href')
        else:
            next_url = urlListElement.get('href')
        return next_url
    return None

<h3>Single Page Test</h3>

In [71]:
def get_soup_from_url(url):
    """Return a BeautifulSoup object from the given URL."""

    options = Options()
    # avoid bot detection  - https://stackoverflow.com/questions/49565042/way-to-change-google-chrome-user-agent-in-selenium/49565254#49565254
    options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_2) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1309.0 Safari/537.17")
    options.add_argument("--start-maximized")  # Maximize the browser window
    options.add_argument("--disable-notifications")
    options.add_argument("--disable-popup-blocking")
    options.add_argument("--disable-infobars")
    options.add_argument("--disable-extensions")
    options.add_experimental_option("prefs", 
                                    {"profile.default_content_setting_values.notifications": 2 
                                    }) 

    driver = webdriver.Chrome(
        # WEB_DRIVER_LOCATION, 
        options=options)
    driver.get(url)

    # Timeout needed for Web page to render (read more about it)
    time.sleep(3)

    html = driver.page_source

    # Get the source of the page and create a BeautifulSoup object
    soup = BeautifulSoup(html, 'html.parser')

    return soup

In [117]:
# single page test code
testURL = "https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&qid=1706467617&rnid=1248985011&ref=sr_nr_selected_filters_p_72_2&ds=v1%3AZRC4RT1%2BRvrPmt8FTw4mkJ%2Bk3F4sePA2BYDE5yVqA0s" 

tb = pt.PrettyTable() 
tb.field_names = ["Title", "Product URL", "Price", "Solder", "Review", "Review Number", "Image URL", "Crawl Time"] 
soup = get_soup_from_url(testURL)
next_url = get_next_page_url(soup)
print(next_url)
ProductListElement = soup.find_all('div', {'cel_widget_id': re.compile('MAIN-SEARCH_RESULTS-\d+'),'class': 'template=SEARCH_RESULTS'})
for item in ProductListElement:
    # get the product information
    title, url, price, solder, review, review_number, image_url, crawl_time = product_extractor(item)
    tb.add_row([title, url, price, solder, review, review_number, image_url, crawl_time]) 
print(tb)

https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&page=2&qid=1706502888&rnid=1248985011&ref=sr_pg_1
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+----------------------+--------------------+---------------+----------------------------------------------------------------+---------------------+
|                                                                                 Title                       

In [73]:
from bs4 import BeautifulSoup

def get_soup_from_driver(driver):
    """Return a BeautifulSoup object from the given driver."""

    html = driver.page_source

    # Get the source of the page and create a BeautifulSoup object
    soup = BeautifulSoup(html, 'html.parser')

    return soup

<h3>Main Entry</h3>

<h4>get urlList</h4>

In [105]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

TIMEOUT = 5
user_agent_list = ['Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; Acoo Browser 1.98.744; .NET CLR 3.5.30729)',
    'Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; Acoo Browser 1.98.744; .NET CLR 3.5.30729)',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; Acoo Browser; GTB5; Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1) ; InfoPath.1; .NET CLR 3.5.30729; .NET CLR 3.0.30618)',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; SV1; Acoo Browser; .NET CLR 2.0.50727; .NET CLR 3.0.4506.2152; .NET CLR 3.5.30729; Avant Browser)',
    'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)',
    'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; GTB5; Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1) ; Maxthon; InfoPath.1; .NET CLR 3.5.30729; .NET CLR 3.0.30618)',
    'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; GTB5;',
    'Mozilla/4.0 (compatible; Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; Acoo Browser 1.98.744; .NET CLR 3.5.30729); Windows NT 5.1; Trident/4.0)',
    'Mozilla/4.0 (compatible; Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; GTB6; Acoo Browser; .NET CLR 1.1.4322; .NET CLR 2.0.50727); Windows NT 5.1; Trident/4.0; Maxthon; .NET CLR 2.0.50727; .NET CLR 1.1.4322; InfoPath.2)',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; Acoo Browser; GTB6; Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1) ; InfoPath.1; .NET CLR 3.5.30729; .NET CLR 3.0.30618)',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; Acoo Browser; GTB5; Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1) ; InfoPath.1; .NET CLR 3.5.30729; .NET CLR 3.0.30618)',
    'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0; GTB6; Acoo Browser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)',
    'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Trident/4.0; Acoo Browser; GTB5; Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1) ; InfoPath.1; .NET CLR 3.5.30729; .NET CLR 3.0.30618)',
    'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)',
    'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; GTB5; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)',
    'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; GTB5; Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1) ; InfoPath.1; .NET CLR 3.5.30729; .NET CLR 3.0.30618)',
    'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Acoo Browser; InfoPath.2; .NET CLR 2.0.50727; Alexa Toolbar)',
    'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Acoo Browser; .NET CLR 2.0.50727; .NET CLR 1.1.4322)',
    'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Acoo Browser; .NET CLR 1.0.3705; .NET CLR 1.1.4322; .NET CLR 2.0.50727; FDM; .NET CLR 3.0.04506.30; .NET CLR 3.0.04506.648; .NET CLR 3.5.21022; InfoPath.2)',
    'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; Acoo Browser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_2) AppleWebKit/537.17 (KHTML, like Gecko) Chrome/24.0.1309.0 Safari/537.17']

options = Options()
    # avoid bot detection  - https://stackoverflow.com/questions/49565042/way-to-change-google-chrome-user-agent-in-selenium/49565254#49565254
options.add_argument("--user-agent={}".format(random.choice(user_agent_list)))
options.add_argument("--start-maximized")  # Maximize the browser window
options.add_argument("--disable-notifications")
options.add_argument("--disable-popup-blocking")
options.add_argument("--disable-infobars")
options.add_argument("--disable-extensions")
    # options.add_argument("--headless") # Runs Chrome in headless mode.
options.add_argument('--cookie=lwa-context=9872941c399f7d905fe29b64a28c533d; session-id=136-4085273-4202024; ubid-main=132-1087777-9092235; sp-cdn="L5Z9:CA"; x-main="yoLGt6tY2Av179LTUtE1B5i0mc3NoEkYCuq9NUs?IrOpfx447fUI@3NOG@j8yx73"; at-main=Atza|IwEBIDJVhPoDUtx-YjlDA48o5m2WQVmwarVBK9N1_sxeXoMZ_PrzJSm7Wh3VXmVUNir7xDSSNijJbg6BIrm0NobFfF1Kd898nZCHaWRhZFAI59IFu5hGxWGXSi-DyXe9MgymkiwgymPEErPtRokjY_U9vEUKIqhF-6ihd4eCZ6ocYWanMOxit6gFb4kv9_07p80dbW7E2UupHcmSSXowXgqtG70yo7g_NBqaeyT4NQOHi31Ogw; sess-at-main="gAG1urrtNUraAE0iBhe6StwiU2cAxc1vN+lUlrcfbjw="; sst-main=Sst1|PQGvR6pn3EMISk7_KeNZhnCMCfikkXSlYGM2tVdEIgqgZ5rV0wyOOZButl2JEXdREqui-evjq4BIJnHWIvYwCNzwBavOFnUJrNNmK4-hSjOKxED4p4Ky_Ca5QtRF4sSzd7NlniB6-JPwdPQJ2xeqtaX_KnYQJbZTQ5A7XjN89PQTyg3qlqjSHko7xYjuLnBd5qKpV6AmlRyrMMq8B2-AhpDT5rckNRMW5MlTLRBoWJr2utSDcw-ynNh8uV46_jaYnP9NyBvVoO4uyXvnHt5RtDINsywpiG_BC3Y2ohTHi2PbgEQ; lc-main=en_US; session-id-time=2082787201l; i18n-prefs=USD; session-token=Bq3AZZtlXHZ8a8V3Pqtn4l0ObZLszxPa1hhMFWsUGuJizpvF+cjTIjJijvDfBEPIe/gqkzTNTjDQ688gfBQ4fi21qcW/L5U9DZogHIOvkJCafJLBay/CRdaGORbhDuFam9RPK9t+gFRpmslV7xpJF9AsX9DR/ElNrt4SjqccFRgbeJj/tIwla3FsCLJbscdi8Xw9fVM0VoQhI2zOoqL/snvk9rEhy9JEdDDS5pC8TaQaBIaiVv3mecHeDVoUM1HA56iu/44GwR7j1fLX22uE8x2HJHyK2tFNF9I2w6SZmfX6ifw918Ky7ekuMjel0x5PYdvxAtSlZAmRxBu5sThp0xm9CaWlEzLygqm7OFD3c7JSU7lskfZYxg==')
options.add_argument("--accept-encoding=gzip, deflate, br")  
options.add_argument("--accept-language=en-US,en;q=0.9")
options.add_argument("--cache-control=no-cache")
options.add_argument("--content-type=text/plain;charset=UTF-8")
options.add_experimental_option("prefs", 
                                    {"profile.default_content_setting_values.notifications": 2 
                                    }) 

driver = webdriver.Chrome(
        # WEB_DRIVER_LOCATION, 
        options=options)
url = "https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&qid=1706467617&rnid=1248985011&ref=sr_nr_selected_filters_p_72_2&ds=v1%3AZRC4RT1%2BRvrPmt8FTw4mkJ%2Bk3F4sePA2BYDE5yVqA0s" 

# get soup from driver
driver.get(url)
soup = get_soup_from_driver(driver)
next_page_link = get_next_page_url(soup)

urlList = []
urlList.append(url)

# get all the urls
try: 
    while next_page_link: 
        print(next_page_link)
        urlList.append(next_page_link)

        # Timeout needed for Web page to render
        time.sleep(random.randrange(2,TIMEOUT))

        soup = get_soup_from_url(next_page_link)
        next_page_link = get_next_page_url(soup)
        
except Exception:
    print("No more pages")


['https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&qid=1706467617&rnid=1248985011&ref=sr_nr_selected_filters_p_72_2&ds=v1%3AZRC4RT1%2BRvrPmt8FTw4mkJ%2Bk3F4sePA2BYDE5yVqA0s', 'https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&page=2&qid=1706500367&rnid=1248985011&ref=sr_pg_1', 'https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&page=3&qid=1706500373&rnid=1248985011&ref=sr_pg_2', 'https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&page=4&qid=1706500383&rnid=1248985011&ref=sr_pg_3', 'https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&page=5&qid=1706500391&rnid=1248985011&ref=sr_pg_4', 'https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&page=6&qid=1706500401&rnid=12489

<h4>crawel by urlList</h4>

In [112]:
# first attempt result 
# urlList = ["https://www.amazon.com/s?k=Kindle+Store&i=digital-text&crid=35BK3MAMZOGFM&sprefix=%2Cdigital-text%2C79&ref=nb_sb_noss",
# 'https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&page=2&qid=1706498884&rnid=1248985011&ref=sr_pg_1',
# 'https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&page=2&qid=1706498884&rnid=1248985011&ref=sr_pg_1',
# 'https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&page=3&qid=1706498890&rnid=1248985011&ref=sr_pg_2',
# 'https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&page=4&qid=1706498899&rnid=1248985011&ref=sr_pg_3',
# 'https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&page=5&qid=1706498910&rnid=1248985011&ref=sr_pg_4',
# 'https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&page=6&qid=1706498920&rnid=1248985011&ref=sr_pg_5',
# 'https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&page=7&qid=1706498930&rnid=1248985011&ref=sr_pg_6',
# 'https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&page=8&qid=1706498938&rnid=1248985011&ref=sr_pg_7',
# 'https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&page=9&qid=1706498947&rnid=1248985011&ref=sr_pg_8',
# 'https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&page=10&qid=1706498956&rnid=1248985011&ref=sr_pg_9',
# 'https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&page=11&qid=1706498965&rnid=1248985011&ref=sr_pg_10',
# 'https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&page=12&qid=1706498973&rnid=1248985011&ref=sr_pg_11',
# 'https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&page=13&qid=1706498984&rnid=1248985011&ref=sr_pg_12',
# 'https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&page=14&qid=1706498994&rnid=1248985011&ref=sr_pg_13',
# 'https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&page=15&qid=1706499004&rnid=1248985011&ref=sr_pg_14',
# 'https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&page=16&qid=1706499014&rnid=1248985011&ref=sr_pg_15',
# 'https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&page=17&qid=1706499024&rnid=1248985011&ref=sr_pg_16',
# 'https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&page=18&qid=1706499033&rnid=1248985011&ref=sr_pg_17',
# 'https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&page=19&qid=1706499042&rnid=1248985011&ref=sr_pg_18',
# 'https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&page=20&qid=1706499053&rnid=1248985011&ref=sr_pg_19',
# 'https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&page=21&qid=1706499062&rnid=1248985011&ref=sr_pg_20',
# 'https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&page=22&qid=1706499072&rnid=1248985011&ref=sr_pg_21',
# 'https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&page=23&qid=1706499080&rnid=1248985011&ref=sr_pg_22',
# 'https://www.amazon.com/s?k=Kindle+Store&i=digital-text&rh=n%3A133140011%2Cp_n_deal_type%3A23566064011&dc&page=24&qid=1706499091&rnid=1248985011&ref=sr_pg_23']

rows = []
for e in urlList:
    time.sleep(random.randrange(2,5))
    soup = get_soup_from_url(e)
    ProductListElement = soup.find_all('div', {'cel_widget_id': re.compile('MAIN-SEARCH_RESULTS-\d+'),'class': 'template=SEARCH_RESULTS'})
    for item in ProductListElement:
    # get the product information
        title, url, price, solder, review, review_number, image_url, crawl_time = product_extractor(item)
        rows.append([title, url, price, solder, review, review_number, image_url, crawl_time])

<h3>Data Store</h3>

In [104]:
# output to csv
import csv
import os
import time

abspath = os.path.abspath('')
isExists = os.path.exists(abspath+'/outputs')
if not isExists:
    os.makedirs(abspath+'/outputs')

file_name = abspath+"/outputs/{}-amazon-crawl.csv".format(time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(time.time())))

with open(file_name, 'w') as file:
    writer = csv.writer(file)
    writer.writerow(["Title", "Product URL", "Price", "Solder", "Review", "Review Number", "Image URL", "Crawl Time"])
    # split the string into a list and write each element in a row
    writer.writerows(rows) 